# scrapping Airbnb website :

## problem statment :
### which cities have the most special-host or higher/lower price?

## About Airbnb: 



### Airbnb is one of the world’s largest marketplaces for unique,
### authentic places to stay and things to do, offering over 7 million accommodations and 40,000 handcrafted activities, 
### all powered by local hosts. An economic empowerment engine, Airbnb has helped millions of hospitality entrepreneurs
### monetize their spaces and their passions while keeping the financial benefits of tourism in their own communities.
### With more than half a billion guest arrivals to date, and accessible in 62 languages across 191 countries and regions,
### Airbnb promotes people-to-people connection, community and trust around the world.

In [1]:
#import libaries 
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver

In [ ]:
# here I used selenim for scarpping the website 

In [8]:
#calling the driver
driver2 = webdriver.Chrome(executable_path='chromedriver')
# open the page by driver 
driver2.get('https://ar.airbnb.com/s/homes?refinement_paths%5B%5D=%2Fhomes&screen_size=large&search_type=section_navigation&click_referer=t%3ASEE_ALL%7Csid%3Aee546d46-da95-43a4-9296-7383bb06b686%7Cst%3AHOME_GROUPING_TOP_REVIEWED&title_type=TOP_REVIEWED_HOMES&rank_mode=top_rated&last_search_session_id=ee546d46-da95-43a4-9296-7383bb06b686')
sleep(2) # Delay execution for a given number of seconds.
SCROLL_PAUSE_TIME = 10

# Get scroll height
last_height = driver2.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver2.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver2.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


# get all page source into html2
html2 = driver2.page_source
driver2.close()# to close driver 


In [9]:
# get page source into soup by beatfulsoup, so I can handle it.
soup2 = BeautifulSoup(html2, 'html.parser')

In [81]:
# creat arraies for specific items that I want.
hotels = []
prices2 = []
period = []
owners = []
cities = []
rates = []

#create for lop to capture the divs that contain all items one by one.
for i in soup2.find_all('div',attrs={"class":"_1mslzuoh"}):
    #grabbing hotel name.
    hotels.append(i.find(name='a')['aria-label'])
    # grabbing the price. 
    prices2.append(i.find(name='span',attrs={'class':"_1p7iugi"}).text)
    # grabbing the period.
    period.append(i.find(name='div',attrs={'class':"_o60r27k"}).text)
    # owner name have deferent class name in html tag, so I defind it towic.
    #Note!( owner name == _kyaywjf or owner name == _ni3nfhi or owner name == no name)
    owner=i.find(name='span',attrs={'class':"_kyaywjf"})
    owner2=i.find(name='span',attrs={'class':"_ni3nfhi"})
    #if owner and owner2 have null value(no name) append '' into owners
    if owner is None or len(owner) == 0:
        if owner2 is None or len(owner2) == 0:
            owners.append('')
        else:
            owners.append(owner2.text)#if owner2 have value append into owners
    else:
        owners.append(owner.text)# if owner have value append into owners
        
    # also city name have deferent class name in html tag, so I defind it towic.
    #Note!( city name == _1q6rrz5 or city name == _1gsklttw)
    city1=i.find(name='div',attrs={'class':"_1q6rrz5"})
    city2=i.find(name='div',attrs={'class':"_1gsklttw"})
    if city1 is None or len(city1) == 0:
        cities.append(city2.text)# if city1 == null value abslutly city2 have, so append city2
    else :
        cities.append(city1.text)# otherwies append city1 

    #Note!( rate valu == _1iku8rd1 or rate value == null)   
    rate=i.find(name='span',attrs={'class':"_1iku8rd1"})   
    if rate is None or len(rate) == 0:
        rates.append('')
    else:
        rates.append(rate.text)     

In [78]:
#creat Data-frame contain all lists we've created above.
airbnb=pd.DataFrame({'hotel':hotels,'price':prices2,'period':period,'owner':owners,
                     'city':cities,'rate':rates})


In [83]:
#cleaning the data from unwanted character.

airbnb['price']=airbnb.price.apply(lambda x: x.replace('السعر:',''))
airbnb['period']=airbnb.period.apply(lambda x: x.replace('السعر:',''))
airbnb['period']=airbnb.period.apply(lambda x: x.replace('$',''))
airbnb['period']=airbnb.period.apply(lambda x: x.replace('SR',''))
airbnb['period']=airbnb.period.apply(lambda x: x.replace('/',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('0',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('1',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('2',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('3',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('4',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('5',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('6',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('7',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('8',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace('9',''))
airbnb['period']=airbnb.period.apply(lambda x:x.replace(',',''))

In [ ]:
#shows the data after cleaning! 

In [87]:
# What a wonderful first 10th rows :) 
airbnb.head(10)

,hotel,price,period,owner,city,rate
0,CHARMING HOUSE Seaside& Pine Forest,‎$155‎,‎‎ ليلة,مضيف متميز,Labenne,4.89
1,Waterfront with extraordinary view,‎$55‎,‎‎ ليلة,مضيف متميز,Kotor,4.80
2,"平江路大儒巷民国建筑,体验苏州老宅@元元的家 《都挺好》取景地",‎$35‎,‎‎ ليلة,مضيف متميز,سوتشو,4.80
3,A BEAUTIFUL LUXURY VILLA IN NORTH ICELAND,‎$194‎,‎‎ ليلة,مضيف متميز,أكوريري,4.76
4,Beautiful flat in Milano center !!!,‎$70‎,‎‎ ليلة,مضيف متميز,Zona 3 · ميلان,4.93
5,BALIAN TREEHOUSE w beautiful pool,‎$83‎,‎‎ ليلة,مضيف متميز,"بالي · Balian Beach, Bali",4.84
6,Dreamy Tropical Tree House,‎$250‎,‎‎ ليلة,مضيف متميز,جزيرة هاواي · Fern Forest,4.85
7,Pettinarihome Campo de FIORI,‎$100‎,‎‎ ليلة,مضيف متميز,ريجولا · روما,4.84
8,Perfectly located Castro,‎$79‎,‎‎ ليلة,مضيف متميز,ذي كاسترو · سان فرانسيسكو,4.92
9,Off-grid itHouse,‎$450‎,‎‎ ليلة,,Pioneertown,4.93


In [88]:
# an amizing tail you'll ever see :) 
airbnb.tail(10)

,hotel,price,period,owner,city,rate
275,"Sunny, Modern room in East Village!",‎SR319‎,‎‎ ليلة,مضيف متميز,Alphabet City · نيويورك,4.76
276,Historic House of 1924 next to Acropolis,‎SR315‎,‎‎ ليلة,مضيف متميز,كوكاكي · أثينا,4.79
277,Luxury room with Taipei 101 View,‎SR244‎,‎‎ ليلة,مضيف متميز,Wenshan · Taipei City,4.89
278,Luxury Self-Contained Coach House,‎SR375‎,‎‎ ليلة,مضيف متميز,جرينوود-كوكس ويل · تورونتو,4.87
279,Big 1floor 4BR 2 toilets 2bath in Shinjuku Shi...,"‎SR1,001‎",‎‎ ليلة,مضيف متميز,منطقة شيبويا · Shibuya-ku,4.94
280,Experience a houseboat in Amsterdam,‎SR311‎,‎‎ ليلة,,ويستليجكي إيلاندين · أمستردام,4.85
281,Pettinarihome 2 Campo dei Fiori,‎SR375‎,‎‎ ليلة,مضيف متميز,روما سنترو · روما,4.90
282,CASA ALAMEDA LUXURY,‎SR454‎,‎‎ ليلة,,بيمونت ويلشاير · بورتلاند,5.0
283,JF Cozy Apartment DOWNTOWN,‎SR184‎,‎‎ ليلة,مضيف متميز,بورتو,4.78
284,Lisbon Sky View Loft - FREE WIFI,‎SR199‎,‎‎ ليلة,,Mercês · لشبونة,4.72


In [85]:
# the perfict shape 
airbnb.shape

(285, 6)

In [86]:
# to save it 
airbnb.to_csv('airbnb.csv')

---

# kaggle link 

https://www.kaggle.com/wejdanalzahrani/airbnb